In [1]:
import os, code, sys, time, argparse

import numpy as np
from sklearn.neighbors import kneighbors_graph
import tensorflow as tf

import utils
import nn
from utils import REDSHIFTS, PARAMS_SEED, LEARNING_RATE, RS_TAGS

In [2]:
n_P = 32
N = n_P**3
M = 14
k = 3
b = 4
redshifts = [15, 19]

In [3]:
# Load data
X = utils.load_zuni_npy_data(redshifts=redshifts, norm_coo=True)[...,:-1]
Y = X[1] # (1000, N, 6)
X = X[0]

LD: X_0.2280_.npy
LD: X_0.0000_.npy


In [4]:
_x_batch = utils.next_minibatch(X[None,...], b, data_aug=False) # shape (2, b, N, 6)

# split data
x_in    = _x_batch[0] # (b, N, 6)
#x_truth = _x_batch[1] # (b, N, 6)

def graph_get_func(h_in, offset=False, inc_self=False): # for tf.py_func
    return nn.get_kneighbor_list(h_in[...,:3], M, offset_idx=offset, inc_self=inc_self)

# get adj_list
#adj_list = graph_get_func(x_in) # (b, N, M)
#ilist    = nn.alist_to_indexlist(adj_list)
adj_csrs = graph_get_func(x_in)
adj_csrs_offset = graph_get_func(x_in, offset=True)

# get edges (relative dists) and nodes (velocities)
#x_in_edges = nn.get_input_edge_features(x_in, adj_list) # (b, N, M, 3)
#x_in_nodes = nn.get_input_node_features(x_in) # (b, N, 3)

In [5]:
edges = nn.get_input_edge_features_batch(x_in, adj_csrs, M) # (b, N*M, 3)
edges_off = nn.get_input_edge_features_batch_offset(x_in, adj_csrs_offset, M)
np.all(edges == edges_off)

True

In [45]:
def offset_coo_feats(m_in):
    m = np.copy(m_in)
    b = m.shape[0]
    for i in range(b):
        m[i] += i*N
    return m

In [46]:
coo_rows, coo_cols = nn.to_coo_batch(adj_csrs) # (b, N*M)
coo_rows_offset = offset_coo_feats(coo_rows)
coo_cols_offset = offset_coo_feats(coo_cols)

In [51]:
sess = tf.InteractiveSession()

/home/evan/.pyenv/versions/3.6.5/envs/Q/lib/python3.6/site-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [74]:
def usm(x, idx_in, num_seg, offset=False, bcast=False):
    if not offset:
        idx = idx_in + N * np.expand_dims(np.arange(b), axis=1)
        seg_mu = tf.unsorted_segment_mean(x, idx, num_seg)
        #return tf.unsorted_segment_mean(x, b_idx, num_seg).eval()
    else:
        idx = idx_in
        seg_mu = tf.unsorted_segment_mean(x, idx, num_seg)
        #return tf.unsorted_segment_mean(x, idx, num_seg).eval()
    return tf.reshape(seg_mu, [b, N, -1]).eval()
'''
    if bcast:
        seg_mu = tf.gather_nd(x, tf.expand_dims(idx, axis=2))
        return tf.reshape(seg_mu, tf.shape(x)).eval()
    else:
        return tf.reshape(seg_mu, [b, N, -1]).eval()
'''


'\n    if bcast:\n        seg_mu = tf.gather_nd(x, tf.expand_dims(idx, axis=2))\n        return tf.reshape(seg_mu, tf.shape(x)).eval()\n    else:\n        return tf.reshape(seg_mu, [b, N, -1]).eval()\n'

In [75]:
pool_rows = usm(edges, coo_cols, b*N, bcast=True)
pool_rows_off = usm(edges, coo_cols_offset, b*N, offset=True, bcast=True)

In [59]:
pool_rows # (b, N, k)

array([[-0.0083453 , -0.00777651, -0.00649213],
       [ 0.00063422,  0.00049963,  0.00145065],
       [ 0.0005162 , -0.00132297,  0.00078389],
       ...,
       [-0.00273841,  0.00151067,  0.00600649],
       [ 0.00547061,  0.00369451, -0.00135117],
       [-0.00511382,  0.00562881,  0.00616319]], dtype=float32)

In [60]:
pool_rows_off

array([[-0.0083453 , -0.00777651, -0.00649213],
       [ 0.00063422,  0.00049963,  0.00145065],
       [ 0.0005162 , -0.00132297,  0.00078389],
       ...,
       [-0.00273841,  0.00151067,  0.00600649],
       [ 0.00547061,  0.00369451, -0.00135117],
       [-0.00511382,  0.00562881,  0.00616319]], dtype=float32)

In [77]:
np.allclose(pool_rows, pool_rows_off) # True

True

In [79]:
pool_cols = usm(edges, coo_rows, b*N)
pool_cols_off = usm(edges, coo_rows_offset, b*N, offset=True)
np.allclose(pool_cols, pool_cols_off)

True

In [80]:
gather_idx = np.expand_dims(coo_cols_offset, axis=2) # (b, N*M, 1), gather_nd indexes into the first idx.shape[-1] dims

In [82]:
pool_rows_off_bcast = tf.gather_nd(pool_rows_off, gather_idx).eval()

ResourceExhaustedError: OOM when allocating tensor with shape[4,458752,32768,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: GatherNd_1 = GatherNd[Tindices=DT_INT32, Tparams=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](GatherNd_1/params, GatherNd_1/indices/_39)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: GatherNd_1/_41 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_7_GatherNd_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'GatherNd_1', defined 

In [84]:
pool_cols.shape

(4, 32768, 3)

In [93]:
a0 = adj_csrs[0]
a0.indices.shape

(458752,)

In [94]:
a0_coo = a0.tocoo()
a0_coo.row.shape

(458752,)

In [99]:
from sklearn.neighbors import kneighbors_graph, radius_neighbors_graph

In [100]:
x0 = x_in[0,:,:3]
k0 = kneighbors_graph(x0, M, include_self=False)
r0 = radius_neighbors_graph(x0, 0.03, include_self=True)

In [102]:
r0.data.shape

(1380024,)

In [103]:
k0.data.shape

(458752,)

In [105]:
r0.tocoo().row.shape

(1380024,)

In [106]:
r0.data.shape

(1380024,)